In [1]:
!pip install unidecode

In [2]:
from __future__ import annotations
import string
from collections.abc import Collection
from random import randint
import re
from unidecode import unidecode

# Tokenization Steps

In this exercise, you'll code your own tokenizer from scratching using base
Python!

You might normally start with a pretrained tokenizer, but this exercise will
help you get to know see some of the tokenization steps better.

## Define Sample Text

Let's first define some sample text you will use to test your tokenization
steps.

In [3]:
sample_text = '''Mr. Louis continued to say, "Penguins are important, 
but we mustn't forget the nuumber 1 priority: the READER!"
'''

print(sample_text)

Mr. Louis continued to say, "Penguins are important, 
but we mustn't forget the nuumber 1 priority: the READER!"



## Normalization

This step is where you'll normalize your text by converting to lowercase,
removing accented characters, etc.

For example, the text:
```
Did Uncle Max like the jalapeño dip?
```
might be normalized to:
```
did uncle max like the jalapeno dip
```

In [4]:
def normalize_text(text: str) -> str:
    # TODO: Normalize incoming text; can be multiple actions
    normalized_text = unidecode((re.sub(rf"[^\w\s{string.punctuation}]+", "" , text.lower())))
    return normalized_text

In [5]:
# Test out your normalization
normalize_text(sample_text)

'mr. louis continued to say, "penguins are important, \nbut we mustn\'t forget the nuumber 1 priority: the reader!"\n'

## Pretokenization

This step will take in the normalized text and pretokenize the text into a list
of smaller pieces.

For example, the text:
```
Did Uncle Max like the jalapeño dip?
```
might be normalized & then pretokenized to:
```
[
    'did',
    'uncle',
    'max',
    'like',
    'the',
    'jalapeno',
    'dip?',
]
```

In [6]:
def pretokenize_text(text: str) -> list[str]:
    # TODO: Pretokenize normalized text
    # Surrond newline characters with spaces to isolate them as tokens
    text = text.replace("\n", " \n ")
    
    smaller_pieces = re.split(r"[ \t]+", text)
    # Remove empty strings that might appear due to consecutive spaces/tabs
    smaller_pieces = [tok for tok in smaller_pieces if tok]  
    return smaller_pieces

In [7]:
# Test out your pretokenization step (after normalizing the text)
normalized_text = normalize_text(sample_text)
pretokenize_text(normalized_text)

['mr.',
 'louis',
 'continued',
 'to',
 'say,',
 '"penguins',
 'are',
 'important,',
 '\n',
 'but',
 'we',
 "mustn't",
 'forget',
 'the',
 'nuumber',
 '1',
 'priority:',
 'the',
 'reader!"',
 '\n']

## Tokenization

This step will take in the list of pretokenized pieces (after the text has 
been normalized) into the tokens that will be used.

For example, the text:
```
Did Uncle Max like the jalapeño dip?
```
might be normalized, pretokenized, and then tokenized to:
```
[
    'did',
    'uncle',
    'max',
    'like',
    'the',
    'jalapeno',
    'dip'
    '?',
]
```

In [8]:
# Combine normalization and pretokenization steps before breaking things further
def tokenize_text(text: str) -> list[str]:
    # Apply normalization & pretokenization steps 
    normalized_text: str = normalize_text(text)
    pretokenized_text: list[str] = pretokenize_text(normalized_text)
    # TODO: Go through pretokenized text to create a list of tokens
    tokens = []
    for word in pretokenized_text:
        tokens.extend(re.findall(rf"\w+|[{string.punctuation}\n]", word))
    return tokens

In [9]:
# Test out your tokenization (that uses normalizing & pretokenizing functions)
tokenize_text(sample_text)

['mr',
 '.',
 'louis',
 'continued',
 'to',
 'say',
 ',',
 '"',
 'penguins',
 'are',
 'important',
 ',',
 '\n',
 'but',
 'we',
 'mustn',
 "'",
 't',
 'forget',
 'the',
 'nuumber',
 '1',
 'priority',
 ':',
 'the',
 'reader',
 '!',
 '"',
 '\n']

## Postprocessing

This final step will take in the list of tokens from the original text and add
any special tokens to the text.

For example, the text:
```
Did Uncle Max like the jalapeño dip?
```
might be normalized, pretokenized, and then tokenized to:
```
[
    '[BOS]',
    'did',
    'uncle',
    'max',
    'like',
    'the',
    'jalapeno',
    'dip'
    '?',
    '[EOS]',
]
```

In [10]:
def postprocess_tokens(tokens: list[str]) -> list[str]:
    # TODO: Add beginning and end of sequence tokens to your tokenized text
    # Can use a format like '[BOS]' & '[EOS]'
    updated_tokens = ["[BOS]"] + tokens + ["[EOS]"]
    return updated_tokens

In [11]:
# Test full pipeline (normalizing, pretokenizing, tokenizing, & postprocessing)
tokens = tokenize_text(sample_text)
tokens = postprocess_tokens(tokens)

print(tokens)

['[BOS]', 'mr', '.', 'louis', 'continued', 'to', 'say', ',', '"', 'penguins', 'are', 'important', ',', '\n', 'but', 'we', 'mustn', "'", 't', 'forget', 'the', 'nuumber', '1', 'priority', ':', 'the', 'reader', '!', '"', '\n', '[EOS]']


# Encoding & Decoding

## Encoding Text to Token IDs

Create an encoder (`encode()`) that will encode the token strings to integer IDs
by defining how to map each token to a unique ID.

> HINT: 
> 
> An easy method is to assign an arbitrary integer to each unique token from 
> the corpus by iterating through the unique tokens.

In [12]:
# Sample corpus (normally this would be much bigger)
sample_corpus = (
    '''Mr. Louis continued to say, "Penguins are important, \nbut we mustn't forget the nuumber 1 priority: the READER!"''',
    '''BRUTUS:\nHe's a lamb indeed, that baes like a bear.''',
    '''Both by myself and many other friends:\mBut he, his own affections' counsellor,\nIs to himself--I will not say how true--\nBut to himself so secret and so close,'''
)

In [13]:
# TODO: Create an encoder to transform token strings to IDs using the sample
def create_vocab(corpus: Collection[str]):
    tokens = []
    for document in corpus:
        tokens.extend(postprocess_tokens(tokenize_text(document)))
        
    unique_tokens = sorted(set(tokens))
    id_to_token = {i: token for i, token in enumerate(unique_tokens)}
    vocab = {token: i for i, token in enumerate(unique_tokens)}
    
    return id_to_token, vocab

id_to_token, vocab = create_vocab(sample_corpus)
      
def encode(tokens: list[str]) -> list[int]:
    # TODO: Complete this function to encode tokens to integer IDs
    encoded_tokens = [vocab[token] for token in tokens]
    return encoded_tokens


### Test `encode()`

In [14]:
# Use sample text for testing
sample_text = sample_corpus[0]
# Create tokens (to be fed to encode())
tokens = tokenize_text(sample_text)
tokens = postprocess_tokens(tokens)
print(f'Tokens:\n{tokens}\n')

Tokens:
['[BOS]', 'mr', '.', 'louis', 'continued', 'to', 'say', ',', '"', 'penguins', 'are', 'important', ',', '\n', 'but', 'we', 'mustn', "'", 't', 'forget', 'the', 'nuumber', '1', 'priority', ':', 'the', 'reader', '!', '"', '[EOS]']



In [15]:
# Test encode()
encoded_tokens = encode(tokens)
print(f'Encoded Tokens:\n{encoded_tokens}\n')

Encoded Tokens:
[9, 39, 6, 36, 22, 56, 50, 4, 2, 46, 14, 31, 4, 0, 19, 58, 40, 3, 53, 24, 55, 43, 7, 47, 8, 55, 48, 1, 2, 10]



## Decoding Token IDs to Text

Based on your enocder you created (`encode()`), create a decoder (`decode()`) to
take a list of token IDs and map them to their associated token.

In [16]:
# TODO: Create an encoder to transform IDs (from encode()) to token strings

def decode(ids: list[int]) -> list[str]:
    # TODO: Complete this function to decode integer IDs to token strings
    token_strings = [id_to_token[id] for id in ids]
    return token_strings

### Test `decode()`

In [17]:
# Use sample text for testing
sample_text = sample_corpus[0]
# Create tokens
tokens = tokenize_text(sample_text)
tokens = postprocess_tokens(tokens)
print(f'Tokens:\n{tokens}\n')

# Create token IDs (to be fed to decode())
encoded_tokens = encode(tokens)
print(f'Encoded Tokens:\n{encoded_tokens}\n')

Tokens:
['[BOS]', 'mr', '.', 'louis', 'continued', 'to', 'say', ',', '"', 'penguins', 'are', 'important', ',', '\n', 'but', 'we', 'mustn', "'", 't', 'forget', 'the', 'nuumber', '1', 'priority', ':', 'the', 'reader', '!', '"', '[EOS]']

Encoded Tokens:
[9, 39, 6, 36, 22, 56, 50, 4, 2, 46, 14, 31, 4, 0, 19, 58, 40, 3, 53, 24, 55, 43, 7, 47, 8, 55, 48, 1, 2, 10]



In [18]:
# Test out decode()
decoded_tokens = decode(encoded_tokens)
print(f'Decoded Tokens:\n{decoded_tokens}\n')

Decoded Tokens:
['[BOS]', 'mr', '.', 'louis', 'continued', 'to', 'say', ',', '"', 'penguins', 'are', 'important', ',', '\n', 'but', 'we', 'mustn', "'", 't', 'forget', 'the', 'nuumber', '1', 'priority', ':', 'the', 'reader', '!', '"', '[EOS]']



In [19]:
decoded_tokens == tokens

True